In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys


In [4]:
class CPageLoader:
    web_driver = r'C:\Users\acer\Desktop\chromedriver.exe'
    def __init__(self,url):
        chrome_options = Options()
        chrome_options.add_argument("start-maximized")
        chrome_options.add_argument("--diable-extensions")
        self.driver = webdriver.Chrome(self.web_driver,options = chrome_options)
        self.driver.get(url)
        #self.driver.implicitly_wait(3)
        
    def get_cpage_rows(self):
        return self.driver.find_elements_by_class_name('cRow')
    
    def get_cpage_basic_rows(self):
        actual_rows =[]
        advance_component =['accordion','body','bar chart','carousel','search',
                            'search result','contact us','dynamic_cmp_detail',
                            'dynamic_cmp_list','google map','iFrame','image slider',
                            'imagebanner','line chart','login status','logo slider',
                            'managepages','menu','menupages','milestone','pie chart',
                            'Policy Info','row separator','share page','user registration',
                            'siteheader','siteheadermenu','skill bar','SoundCloudPlayer',
                            'subscribe','forgot password','login form','password recover','',
                            'tabs','Text editor','unsubscribe form','verticalcollector',
                            'verticalcollectors','videobanner','layer slider','dynamic form']
        try:
            rows = self.get_cpage_rows()
            for row in rows:
                row_data_type = row.get_attribute('data-type')
                if row_data_type not in advance_component:
                    actual_rows.append(row)
        except Exception as e:
            raise e
        return actual_rows
    
    def get_cpage_cols(self,row_obj):
        return row_obj.find_elements_by_xpath('div/div')
    
    def get_data_type(self,obj):
        return obj.get_attribute('data-type')
    
    def get_property(self,class_dict=None,class_attribute=None):
        ATTRIBUTE_CLASS = {
                            'margin-left':'Ml',
                            'margin-top':'Mt',
                            'margin-right':'Mr',
                            'margin-bottom':'Mb',
                            'padding-left':'Pl-',
                            'padding-right':'Pr-',
                            'padding-bottom':'Pb-',
                            'padding-top':'Pt-',
                            'text-align':'TxAl-',
                            'font-size':'Fs-',
                            'line-height':'Lh-',
                            'letter-space':'LtrSpc-',
                            'text-decoration':'Td',
                            'font-style':'FsI',
                            'font-weight':'f-weight-',
                            'width':'sfCol_',
                            'height': 'H-',
                            'font-family':'ff-',
                            'text-transform':'tx',
                            'width':'W-',
                            'font-icon':'fa-'
                            
        }
        a_c = class_dict if class_dict is not None else ATTRIBUTE_CLASS
        property_data = {}
        comp_class = class_attribute
        classes = comp_class.split()
        for _class in classes:
            for i in a_c:
                if _class.startswith(a_c[i]):

                    value = None

                    if _class.startswith('M'):
                        if _class[2] == 'N':
                            value = _class[_class.index('-'):]
                        else:
                            value = _class[_class.index('-')+1:]
                    elif _class.startswith('f-weight-'):
                        value = _class[_class.index('t-')+2:]

                    elif _class.startswith('TxAl'):
                        allign = _class[_class.index('-')+1:]
                        if allign == 'l':
                            value = 'left'
                        elif allign == 'c':
                            value = 'center'
                        elif allign == 'r':
                            value = 'right'
                        elif allign == 'n':
                            value = 'none'

                    elif _class.startswith('Td'):
                        if _class == 'TdU':
                            value = 'underline'
                        if _class == 'TdS':
                            value = 'line-through'

                    elif _class == 'FsI':
                        value = 'italic'
                    elif _class.startswith('sfCol_') or _class.startswith('W-'):
                        if _class.startswith('sfCol_'):
                            value = _class[_class.index('_')+1:]
                        else:
                            value = _class[_class.index('-')+1:]
                    elif _class.startswith('tx'):
                        if _class == 'txU':
                            value = 'uppercase'
                        elif _class == 'txL':
                            value = 'lowwercase'
                        elif _class == 'txC':
                            value = 'capitalized'
                    else:
                        value = _class[_class.index('-')+1:]

                    temp = value
                    property_data[i] = temp
        return property_data
    
    def get_page_data(self):

        comp_data = []
        undefined_data = []

        grp_data_component = ['container','holder']
        data_types = ['button','font icon','image','icon text','rich text','social link',
                        'text link','text','youtube video','Image Link','heading','form','underline']
        rows = self.get_cpage_basic_rows()
        for row in rows:
            row_data = []
            #row property
            row_property_class = row.get_attribute('class')
            row_property_data =  self.get_property(class_attribute=row_property_class)
            columns = self.get_cpage_cols(row)
            for column in columns:
                column_data =[]
                #column_property
                column_property_class = column.get_attribute('class')
                column_property_data = self.get_property(class_attribute=column_property_class)
                div_class = column.get_attribute('class')
                if div_class == 'cGrid ':
                    divisions = column.find_elements_by_xpath('div/div')
                else:
                    divisions = column.find_elements_by_xpath('div')
                    
                for division in divisions:
                    data_type = self.get_data_type(division)
                    if data_type in grp_data_component:
                        if data_type == grp_data_component[0]:
                            container_property_class = division.get_attribute('class')
                            container_property_data = self.get_property(class_attribute=container_property_class)
                            container_data = self.get_container_data(division)
                            column_data.append({data_type:{'container_property':container_property_data,'container_data':container_data}})
                        else:
                            #holder property
                            holder_property_class = division.get_attribute('class')
                            holder_property_data = self.get_property(class_attribute=holder_property_class)
                            holder_data = self.get_holder_data(division)
                            column_data.append({data_type:{'holder_property':holder_property_data,'holder_data':holder_data}})
                    elif data_type in data_types:
                        data = self.get_component_data(division)
                        column_data.append({data_type:data})
                    else:
                        column_data.append({'unefined':division.get_attribute('class'),'data_type':data_type})
                        
                row_data.append({'column':{'column_property':column_property_data,'column_data':column_data}})
            
            
            comp_data.append({'row':{'row_property':row_property_data,'row_data':row_data}})
        return comp_data
    def get_container_data(self,container_obj):
        cont_data = []
        undefined_data = []
        data_types = ['button','font icon','image','icon text','rich text','social link',
                        'text link','text','youtube video','Image Link','heading','form','underline']
        
        grp_data_component = ['holder']
        columns = self.get_cpage_cols(container_obj)
        for column in columns:
            column_data =[]
            divisions = column.find_elements_by_xpath('div')
            for div in divisions:
                data_type = self.get_data_type(div)
                if data_type in grp_data_component:
                    holder_property_class = div.get_attribute('class')
                    holder_property_data = self.get_property(class_attribute=holder_property_class)
                    holder_data = self.get_holder_data(div)
                    column_data.append({data_type:{'holder_property':holder_property_data,'holder_data':holder_data}})
                elif data_type in data_types:
                    data = self.get_component_data(div)
                    column_data.append({data_type:data})
                else:
                    column_data.append({'unefined':div.get_attribute('class'),'data_type':data_type})
            cont_data.append({'column':column_data})
        return cont_data
    
    def get_holder_data(self,holder_obj):
        
        undefined_data= []
        data = {}
        column_data = []
        data_types = ['button','font icon','image','icon text','rich text','social link',
                        'text link','text','youtube video','Image Link','heading','form','underline']
        
        grp_data_component = ['holder']
        data_div = self.get_cpage_cols(holder_obj)
        
        for div in data_div:
            data_type = self.get_data_type(div)
            if data_type in grp_data_component:
                holder_property_class = div.get_attribute('class')
                holder_property_data = self.get_property(class_attribute=holder_property_class)
                holder_data = self.get_holder_data(div)
                column_data.append({data_type:{'holder_property':holder_property_data,'holder_data':holder_data}})
            elif data_type in data_types:
                component_data = self.get_component_data(div)
                column_data.append({data_type:component_data})
            else:
                column_data.append({'unefined':div.get_attribute('class'),'data_type':data_type})
        return column_data
    
    def get_data(self,data_obj):
        pass
        
    def get_component_data(self,component_obj):
        
        data = []
        data_type = self.get_data_type(component_obj)
        
        if data_type == 'text':
            data =self.get_text_property(component_obj)
        elif data_type == 'heading':
            data = self.get_heading_property(component_obj)
        elif data_type == 'image':
            data = self.get_image_property(component_obj)
        elif data_type == 'underline':
            data = self.get_underline_property(component_obj)
        elif data_type == 'rich text':
            data = self.get_rich_text_property(component_obj)
        elif data_type == 'text link':
            data = self.get_text_link_property(component_obj)
        elif data_type == 'Image Link':
            data = self.get_image_link_property(component_obj)
        elif data_type == 'button':
            data = self.get_button_property(component_obj)
        elif data_type == 'font icon':
            data = self.get_font_icon_property(component_obj)
        elif data_type == 'icon text':
            data = self.get_icon_text_property(component_obj)
        elif data_type == 'social link':
            data = self.get_social_link_property(component_obj)
        elif data_type == 'youtube video':
            data = self.get_youtube_video_property(component_obj)
        elif data_type == 'form':
            #data = self.get_form_property(component_obj)
            data = {}
        return data

    
    def get_form_property(self,form_obj):
        return 'form'
    
    def get_youtube_video_property(self,youtube_video_obj):
        
        data = {}
        component_property_data = {}
        youtube_video_property_data = {}

        component_property_class = youtube_video_obj.get_attribute('class')
        component_property_data = self.get_property(class_attribute=component_property_class)

        youtube_video_div = youtube_video_obj.find_element_by_xpath('div')
        youtube_video_property_class = youtube_video_div.get_attribute('class')
        youtube_video_property_data = self.get_property(class_attribute=youtube_video_property_class)
        
        video_link = youtube_video_div.find_element_by_xpath('iframe').get_attribute('src')
        data = {
            'component_property':component_property_data,
            'youtube_video_property':youtube_video_property_data,
            'youtube_link':video_link
        }
        return data

    def get_social_link_property(self,social_link_obj):
        
        data = {}
        component_property_data ={}
        social_link_property_data = []

        component_property_class = social_link_obj.get_attribute('class')
        social_link_list = social_link_obj.find_elements_by_xpath('div/div')

        component_property_data = self.get_property(class_attribute=component_property_class)
        for social_link in social_link_list:
            icon_data = self.get_font_icon_property(social_link)
            social_link_property_data.append(icon_data)
        
        data = {
            'component_property':component_property_data,
            'social_links':social_link_property_data
        }

        return data




    def get_icon_text_property(self,icon_text_obj):
        data = {}
        component_property_data = {}
        icon_text_property_data = {}
        icon_text = []
        component_property_class = icon_text_obj.get_attribute('class')
        icon_text_div = icon_text_obj.find_element_by_xpath('ul')
        icon_text_property_class = icon_text_div.get_attribute('class')
        
        component_property_data = self.get_property(class_attribute = component_property_class)
        icon_text_property_data = self.get_property(class_attribute = icon_text_property_class )
        
        show_icon = True if icon_text_div.get_attribute('data-showicon')=='true' else False
        icon_text_property_data['show-icon'] = show_icon
        
        list_items = icon_text_div.find_elements_by_xpath('li')
        

        for items in list_items:

            item_property_data = {}
            icon_property_data = {}
            label_property_data = {}

            item_property_class = items.get_attribute('class')
            label_property_class = items.find_element_by_xpath('label').get_attribute('class')

            item_property_data = self.get_property(class_attribute=item_property_class)
            label_property_data = self.get_property(class_attribute=label_property_class)
            if show_icon:
                icon_property_class = items.find_element_by_xpath('i').get_attribute('class')
                icon_property_data = self.get_property(class_attribute=icon_property_class)
            label_text = items.find_element_by_xpath('label').text
            list_data = {
                'list_item_property':item_property_data,
                'icon_property':icon_property_data,
                'label_property':label_property_data,
                'label_text':label_text
            }
            
            icon_text.append(list_data)




        
        
        data = {'component_property':component_property_data,
               'icon_text_property':icon_text_property_data,
               'icon_text':icon_text}
        return data

    def get_font_icon_property(self,font_icon_obj):
        data = {}
        component_property_data = {}
        font_icon_property_data = {}
        component_property_class = font_icon_obj.get_attribute('class')
        font_icon_div = None
        try:
            font_icon_div = font_icon_obj.find_element_by_xpath('a')
            font_icon_property_class = font_icon_div.get_attribute('class')
            font_icon_property_data = self.get_property(class_attribute = font_icon_property_class )
            font_icon_property_data['data-href'] = font_icon_div.get_attribute('data-href')

        except:
            font_icon_div = font_icon_obj.find_element_by_xpath('div')
            font_icon_property_class = font_icon_div.get_attribute('class')
            font_icon_property_data = self.get_property(class_attribute = font_icon_property_class )
        
        component_property_data = self.get_property(class_attribute = component_property_class)
        
        font_icon_i = font_icon_div.find_element_by_xpath('i').get_attribute('class')
        font_icon = self.get_property(class_attribute=font_icon_i)



        
        
        data = {'component_property':component_property_data,
               'font_icon_property':font_icon_property_data,
               'font_icon':font_icon}
        return data

    def get_button_property(self,button_obj):
        data = {}
        component_property_data = {}
        button_property_data = {}
        component_property_class = button_obj.get_attribute('class')
        button_div = button_obj.find_element_by_xpath('a')
        button_property_class = button_div.get_attribute('class')
        
        component_property_data = self.get_property(class_attribute = component_property_class)
        button_property_data = self.get_property(class_attribute = button_property_class )
        action_type = button_div.get_attribute('data-actiontype')
        button_property_data['actiontype'] = action_type
        if action_type == 'Actlink':
            data_link = button_div.get_attribute('data-link')
            data_href = button_div.get_attribute('data-href')
            button_property_data['data-link'] = data_link
            button_property_data['data-href'] = data_href
        elif action_type == 'ActflDownload':
            data_attachment = button_div.get_attribute('data-attachment')
            button_property_data['data-attachment'] = data_attachment
        elif action_type == 'ActSection':
            section_id = button_div.get_attribute('sectionid')
            button_property_data['section-id'] = section_id
        elif action_type == 'ActPopup':
            button_propery_data['pop-up'] = 'not-specific'
        button_text = button_div.find_element_by_xpath('span').text

        
        data = {'component_property':component_property_data,
               'button_property':button_property_data,
               'button':button_text}
        return data
        
    def get_image_link_property(self,image_link_obj):
        data = {}
        component_property_data = {}
        image_property_data = {}
        component_property_class = image_link_obj.get_attribute('class')
        image_link_div = image_link_obj.find_element_by_xpath('a')
        image_link_property_class = image_link_div.get_attribute('class')
        link_type = image_link_div.get_attribute('data-link')
        
        component_property_data = self.get_property(class_attribute = component_property_class)
        image_link_property_data = self.get_property(class_attribute = image_link_property_class )
        image_link_property_data.update(dict(image_link_div.size))
        image_link_property_data['link_type'] = link_type
        image_link_property_data['redirect_link'] = image_link_div.get_attribute('href')
        
        image = image_link_div.find_element_by_xpath('img')
        image_link = image.get_attribute('src')
        
        data = {'component_property':component_property_data,
               'image_link_property':image_link_property_data,
               'image_link':image_link}
        return data
        
    def get_text_link_property(self,text_link_obj):
        
        data = {}
        component_property_data = {}
        text_property_data = {}
        component_property_class = text_link_obj.get_attribute('class')
        text_link_div = text_link_obj.find_element_by_xpath('div/a')
        text_link_property_class = text_link_div.get_attribute('class')
        link_type = text_link_div.get_attribute('data-link')
        
        component_property_data = self.get_property(class_attribute = component_property_class)
        text_link_property_data = self.get_property(class_attribute = text_link_property_class )
        text_link_property_data['link_type'] = link_type
        
        data = {'component_property':component_property_data,
               'rich_text_property':text_link_property_data,
               'text_link':text_link_div.get_attribute('href')}
        return data
        
    def get_rich_text_property(self,rich_text_obj):
        data = {}
        component_property_data = {}
        text_property_data = {}
        component_property_class = rich_text_obj.get_attribute('class')
        rich_text_div = rich_text_obj.find_element_by_xpath('div/div/div')
        rich_text_property_class = rich_text_div.get_attribute('class')
        text_font_family = rich_text_div.value_of_css_property('font-family')
        try:
            quoted = rich_text_div.find_element_by_xpath('blockquote')
            if quoted is not None:
                is_quoted = True
            else:
                is_quoted = False   
        except:
            is_quoted = False
        component_property_data = self.get_property(class_attribute = component_property_class)
        rich_text_property_data = self.get_property(class_attribute = rich_text_property_class )
        rich_text_property_data['font-family'] = text_font_family
        rich_text_property_data['blockquote'] = is_quoted
        data = {'component_property':component_property_data,
               'rich_text_property':rich_text_property_data,
               'text':rich_text_div.get_attribute('innerText')}
        return data
    def get_text_property(self,text_obj):
        
        data = {}
        component_property_data = {}
        text_property_data = {}
        component_property_class = text_obj.get_attribute('class')
        text_div = text_obj.find_element_by_xpath('p')
        text_property_class = text_div.get_attribute('class')
        component_property_data = self.get_property(class_attribute = component_property_class)
        text_property_data = self.get_property(class_attribute = text_property_class )
        
        data = {'component_property':component_property_data,
               'text_property':text_property_data,
               'text':text_div.get_attribute('innerHTML')}
        
        return data
    
    def get_heading_property(self,heading_obj):
        
        data = {}
        component_property_data = {}
        heading_property_data = {}
        component_property_class = heading_obj.get_attribute('class')
        heading_div = heading_obj.find_element_by_xpath('h1')
        heading_property_class = heading_div.get_attribute('class')
        component_property_data = self.get_property(class_attribute = component_property_class)
        heading_property_data = self.get_property(class_attribute = heading_property_class )
        
        data = {'component_property':component_property_data,
               'heading_property':heading_property_data,
               'heading':heading_div.get_attribute('innerHTML')}
        
        return data
        
        
    def get_image_property(self,image_obj):

        data = {}
        component_property_data = {}
        image_property_data = {}
        
        component_property_class = image_obj.get_attribute('class')
        image_div = image_obj.find_element_by_xpath('img')
        component_property_data = self.get_property(class_attribute = component_property_class)
        image_property_data = image_div.size

        data = {'component_property':component_property_data,
               'image_property':image_property_data,
               'image_src':image_div.get_attribute('src')}

        return data
    
    def get_underline_property(self,underline_obj):
        
        data = {}
        component_property_data = {}
        underline_property_data = {}
        
        component_property_class = underline_obj.get_attribute('class')
        
        underline_div = underline_obj.find_elements_by_tag_name('div')
        underline_property_class = underline_div[0].get_attribute('class')
        
        component_property_data = self.get_property(class_attribute = component_property_class)
        underline_property_data = self.get_property(class_attribute = underline_property_class )
        
        data = {'component_property':component_property_data,
                'underline_property':underline_property_data,
               'underline':'underline'
               }
        return data

    

        
scrapper = CPageLoader('https://delasheshstha.contentder.com/')
print(scrapper.get_page_data())


[{'row': {'row_property': {'text-align': 'center'}, 'row_data': [{'column': {'column_property': {'padding-top': '0', 'padding-right': '0', 'padding-bottom': '0', 'padding-left': '0', 'text-align': None}, 'column_data': [{'holder': {'holder_property': {}, 'holder_data': [{'youtube video': {'component_property': {}, 'youtube_video_property': {}, 'youtube_link': 'https://www.youtube.com/embed/_YsUWcUvR9c?controls=0&autoplay=0'}}]}}]}}]}}]
